In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from datetime import datetime, timedelta
import sweetviz as sv
%matplotlib inline

In [2]:
# read data from csv file.
df = pd.read_csv('../data/archive/weather-sa-2017-2019-clean.csv')
# df = pd.DataFrame(data)
df.head()

,Unnamed: 0,city,date,time,year,month,day,hour,minute,weather,temp,wind,humidity,barometer,visibility
0,0,Qassim,1 January 2017,00:00,2017,1,1,24,0,Clear,17,11,64%,1018.0,16
1,1,Qassim,1 January 2017,01:00,2017,1,1,1,0,Clear,17,6,64%,1018.0,16
2,2,Qassim,1 January 2017,03:00,2017,1,1,3,0,Clear,15,11,72%,1019.0,16
3,3,Qassim,1 January 2017,04:00,2017,1,1,4,0,Clear,15,11,72%,1019.0,16
4,4,Qassim,1 January 2017,05:00,2017,1,1,5,0,Clear,15,9,72%,1019.0,16


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249023 entries, 0 to 249022
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  249023 non-null  int64  
 1   city        249023 non-null  object 
 2   date        249023 non-null  object 
 3   time        249023 non-null  object 
 4   year        249023 non-null  int64  
 5   month       249023 non-null  int64  
 6   day         249023 non-null  int64  
 7   hour        249023 non-null  int64  
 8   minute      249023 non-null  int64  
 9   weather     249023 non-null  object 
 10  temp        249023 non-null  int64  
 11  wind        249023 non-null  int64  
 12  humidity    249006 non-null  object 
 13  barometer   248951 non-null  float64
 14  visibility  249023 non-null  int64  
dtypes: float64(1), int64(9), object(5)
memory usage: 28.5+ MB


In [4]:
df.shape

(249023, 15)

In [5]:
df.isnull().sum()

Unnamed: 0     0
city           0
date           0
time           0
year           0
month          0
day            0
hour           0
minute         0
weather        0
temp           0
wind           0
humidity      17
barometer     72
visibility     0
dtype: int64

In [6]:
df.drop(['Unnamed: 0', 'time'], axis =1 , inplace =True)
df.head(10)

,city,date,year,month,day,hour,minute,weather,temp,wind,humidity,barometer,visibility
0,Qassim,1 January 2017,2017,1,1,24,0,Clear,17,11,64%,1018.0,16
1,Qassim,1 January 2017,2017,1,1,1,0,Clear,17,6,64%,1018.0,16
2,Qassim,1 January 2017,2017,1,1,3,0,Clear,15,11,72%,1019.0,16
3,Qassim,1 January 2017,2017,1,1,4,0,Clear,15,11,72%,1019.0,16
4,Qassim,1 January 2017,2017,1,1,5,0,Clear,15,9,72%,1019.0,16
5,Qassim,1 January 2017,2017,1,1,6,0,Clear,13,13,82%,1019.0,16
6,Qassim,1 January 2017,2017,1,1,7,0,Sunny,12,7,88%,1019.0,16
7,Qassim,1 January 2017,2017,1,1,8,0,Sunny,14,9,72%,1021.0,16
8,Qassim,1 January 2017,2017,1,1,9,0,Sunny,15,9,72%,1021.0,7
9,Qassim,1 January 2017,2017,1,1,10,0,Sunny,17,7,64%,1021.0,9


In [7]:
# seperating categorical data from numerical
df_cate = df.select_dtypes(include=['object'])
df_cate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249023 entries, 0 to 249022
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   city      249023 non-null  object
 1   date      249023 non-null  object
 2   weather   249023 non-null  object
 3   humidity  249006 non-null  object
dtypes: object(4)
memory usage: 7.6+ MB


In [8]:
# apply Label encoder to df_cate

le = preprocessing.LabelEncoder()
df_cate = df_cate.apply(le.fit_transform)
df_cate.head()

,city,date,weather,humidity
0,10,10,1,62
1,10,10,1,62
2,10,10,1,71
3,10,10,1,71
4,10,10,1,71


In [9]:
df = df.drop(df_cate.columns, axis=1)
df = pd.concat([df, df_cate], axis=1)
df.head()

,year,month,day,hour,minute,temp,wind,barometer,visibility,city,date,weather,humidity
0,2017,1,1,24,0,17,11,1018.0,16,10,10,1,62
1,2017,1,1,1,0,17,6,1018.0,16,10,10,1,62
2,2017,1,1,3,0,15,11,1019.0,16,10,10,1,71
3,2017,1,1,4,0,15,11,1019.0,16,10,10,1,71
4,2017,1,1,5,0,15,9,1019.0,16,10,10,1,71


In [10]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,249013,249014,249015,249016,249017,249018,249019,249020,249021,249022
year,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,2017.0,...,2019.0,2019.0,2019.0,2019.0,2019.0,2019.0,2019.0,2019.0,2019.0,2019.0
month,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
day,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0
hour,24.0,1.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0
minute,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
temp,17.0,17.0,15.0,15.0,15.0,13.0,12.0,14.0,15.0,17.0,...,35.0,34.0,35.0,35.0,34.0,32.0,29.0,27.0,26.0,24.0
wind,11.0,6.0,11.0,11.0,9.0,13.0,7.0,9.0,9.0,7.0,...,19.0,9.0,15.0,19.0,17.0,19.0,9.0,7.0,0.0,7.0
barometer,1018.0,1018.0,1019.0,1019.0,1019.0,1019.0,1019.0,1021.0,1021.0,1021.0,...,1016.0,1015.0,1014.0,1014.0,1014.0,1014.0,1015.0,1016.0,1017.0,1017.0
visibility,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,7.0,9.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,16.0,16.0
city,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249023 entries, 0 to 249022
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   year        249023 non-null  int64  
 1   month       249023 non-null  int64  
 2   day         249023 non-null  int64  
 3   hour        249023 non-null  int64  
 4   minute      249023 non-null  int64  
 5   temp        249023 non-null  int64  
 6   wind        249023 non-null  int64  
 7   barometer   248951 non-null  float64
 8   visibility  249023 non-null  int64  
 9   city        249023 non-null  int32  
 10  date        249023 non-null  int32  
 11  weather     249023 non-null  int32  
 12  humidity    249023 non-null  int32  
dtypes: float64(1), int32(4), int64(8)
memory usage: 20.9 MB


In [13]:
analyze_report = sv.analyze(df)
analyze_report.show_html('analyze.html', )

                                             |                                             | [  0%]   00:00 ->…

Report analyze.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
